In [9]:
# For, Windows, please import pyvips with this cell
# import os
# vipshome = 'c:\\vips-dev-8.7\\bin'
# os.environ['PATH'] = vipshome + ';' + os.environ['PATH']

In [10]:
import pyvips  #must use conda to install
import os
import glob
from natsort import natsorted
from time import time

In [11]:
def ims2ometiff(ims,outputfilepath):
    imobjs = []
    for idx,im in enumerate(ims):
        impth = os.path.join(input_dir, im)
        imobj = pyvips.Image.new_from_file(impth)
        # imobj = pyvips.Image.openslideload(impth,level=0)
        if imobj.hasalpha(): imobj = imobj[:-1]
        imobjs.append(imobj)
        print('image {} has dimensions [{},{}]'.format(idx,imobj.height,imobj.width))
    print('z stack height :',len(imobjs))
    comp = pyvips.Image.arrayjoin(imobjs, across=1)
    image_height = imobj.height
    image_width = imobj.width
    image_bands = imobj.bands
    comp = comp.copy()

    # set minimal OME metadata
    # before we can modify an image (set metadata in this case), we must take a
    # private copy
    comp.set_type(pyvips.GValue.gint_type, "page-height", image_height)
    comp.set_type(pyvips.GValue.gstr_type, "image-description",
                  f"""<?xml version="1.0" encoding="UTF-8"?>
    <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
        xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
        <Image ID="Image:0">
            <!-- Minimum required fields about image dimensions -->
            <Pixels DimensionOrder="XYCZT"
                    ID="Pixels:0"
                    SizeC="{len(imobjs)}"
                    SizeT="1"
                    SizeX="{image_width}"
                    SizeY="{image_height}"
                    SizeZ="1"
                    Type="uint8">
            </Pixels>
        </Image>
    </OME>""")


    #jpeg,jp2k,lzw,
    print('saving image...')
    # jp2k breaks the format somehow
    start=time()
    comp.tiffsave(outputfilepath, compression="none", tile=True,
                  tile_width=512, tile_height=512,
                  pyramid=True, subifd=True)
    print('elapsed time: {}'.format(round(time()-start)))
    print('ome-tiff saved here: ',outputfilepath)

In [12]:
if __name__=='__main__':
    input_dir = '/Volumes/Digital pathology image lib-1/HubMap Skin TMC project/HM-SR1-Skin-P011-B1-SB02'
    output_dir = os.path.join(input_dir, 'cstack')
    if not os.path.exists(output_dir): os.mkdir(output_dir)
    ims = glob.glob(os.path.join(input_dir,'*.tif'))
    ims = natsorted(ims)
    # ims = [_ for _ in ims if 'round02' in _]
    print(len(ims))
    ims2ometiff(ims,os.path.join(output_dir,'channel4.ome.tiff'))


4
image 0 has dimensions [33513,35183]
image 1 has dimensions [33513,35183]
image 2 has dimensions [33513,35183]
image 3 has dimensions [33513,35183]
z stack height : 4
saving image...
elapsed time: 314
ome-tiff saved here:  /Volumes/Digital pathology image lib/Fan Wu/Pei-Hsun/08251 2C_24/well image stitch/1x_/cstack/channel4.ome.tiff
